In [1]:
# importing required packages

import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import cohen_kappa_score
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

/home/linux-device/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/linux-device/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
importing_dataset = pd.read_csv('training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')

# dependent variable
scores = importing_dataset['domain1_score']
dataset = importing_dataset.loc[:,['essay_id', 'essay_set', 'essay', 'domain1_score']]

In [3]:
# Generating word tokens after removing characters other than alphabets, converting them to lower case and
# removing stopwords from the text'''

def word_tokens(essay_text):
    essay_text = re.sub("[^a-zA-Z]", " ", essay_text)
    words = essay_text.lower().split()
    stop_words = set(stopwords.words("english"))
    words = [w for w in words if not w in stop_words]
    return (words)

In [4]:
# Generating sentence tokens from the essay and finally the word tokens

def sentence_tokens(essay_text):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_tokens = tokenizer.tokenize(essay_text.strip())
    sentences = []
    for sent_token in sent_tokens:
        if len(sent_token) > 0:
            sentences.append(word_tokens(sent_token))
    return sentences

In [5]:
# Generating a vector of features

def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

In [6]:
# Generating word vectors to be used in word2vec model

def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay_text in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay_text, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [7]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [8]:
# Applying k-fold cross validation

cv = KFold(len(dataset), n_folds=5, shuffle=True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv:
    print("\n------------Fold {}------------\n".format(count))
    X_test, X_train, y_test, y_train = dataset.iloc[testcv], dataset.iloc[traincv], scores.iloc[testcv], scores.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training set of essays.
            sentences += sentence_tokens(essay)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_text in train_essays:
        clean_train_essays.append(word_tokens(essay_text))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_text in test_essays:
        clean_test_essays.append(word_tokens(essay_text))
    testDataVecs = getAvgFeatureVecs(clean_test_essays, model, num_features)
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    '''Evaluation metric used : 
    1. Mean squared error
    2. Variance
    3. Cohen's kappa score
    Expected results - Minimum error, maximum variance(For variance, best possible score is 1.0, lower 
    values are worse.) and maximum kappa score(1 depicting the best scores)'''
    
    # Mean squared error
    print("Mean squared error: {0:.2f}".format(mean_squared_error(y_test.values, y_pred)))

    # Explained variance score: 1 is perfect prediction
    print('Variance: {0:.2f}'.format(explained_variance_score(y_test.values, y_pred)))  
    
    #Cohen's kappa score
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {0:.2f}".format(result))
    results.append(result)

    count += 1


------------Fold 1------------

Training Word2Vec Model...


/home/linux-device/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10380/10380 [==============================] - 7s 649us/step - loss: 63.7772 - mean_absolute_error: 4.3342
Epoch 2/50
10380/10380 [==============================] - 5s 467us/step - loss: 39.4171 - mean_absolute_error: 3.4872
Epoch 3/50
10380/10380 [========

10381/10381 [==============================] - 5s 446us/step - loss: 29.7156 - mean_absolute_error: 3.3266
Epoch 5/50
10381/10381 [==============================] - 5s 449us/step - loss: 28.4775 - mean_absolute_error: 3.2620
Epoch 6/50
10381/10381 [==============================] - 5s 447us/step - loss: 27.8784 - mean_absolute_error: 3.1805
Epoch 7/50
10381/10381 [==============================] - 5s 451us/step - loss: 25.7029 - mean_absolute_error: 3.0430
Epoch 8/50
10381/10381 [==============================] - 5s 452us/step - loss: 24.7546 - mean_absolute_error: 2.8816
Epoch 9/50
10381/10381 [==============================] - 5s 450us/step - loss: 21.6947 - mean_absolute_error: 2.7223
Epoch 10/50
10381/10381 [==============================] - 5s 453us/step - loss: 19.4678 - mean_absolute_error: 2.5736
Epoch 11/50
10381/10381 [==============================] - 5s 453us/step - loss: 18.0379 - mean_absolute_error: 2.4498
Epoch 12/50
10381/10381 [==============================] - 5s 458

Epoch 15/50
10381/10381 [==============================] - 5s 464us/step - loss: 15.1738 - mean_absolute_error: 2.2102
Epoch 16/50
10381/10381 [==============================] - 5s 463us/step - loss: 14.3682 - mean_absolute_error: 2.1598
Epoch 17/50
10381/10381 [==============================] - 5s 453us/step - loss: 13.2707 - mean_absolute_error: 2.0810
Epoch 18/50
10381/10381 [==============================] - 5s 454us/step - loss: 13.7793 - mean_absolute_error: 2.0955
Epoch 19/50
10381/10381 [==============================] - 5s 453us/step - loss: 12.2840 - mean_absolute_error: 2.0023
Epoch 20/50
10381/10381 [==============================] - 5s 465us/step - loss: 12.1872 - mean_absolute_error: 1.9802
Epoch 21/50
10381/10381 [==============================] - 5s 456us/step - loss: 11.3628 - mean_absolute_error: 1.9252
Epoch 22/50
10381/10381 [==============================] - 5s 464us/step - loss: 11.9368 - mean_absolute_error: 1.9374
Epoch 23/50
10381/10381 [=======================

10381/10381 [==============================] - 5s 495us/step - loss: 10.7070 - mean_absolute_error: 1.8209
Epoch 26/50
10381/10381 [==============================] - 5s 492us/step - loss: 9.9841 - mean_absolute_error: 1.7700
Epoch 27/50
10381/10381 [==============================] - 5s 499us/step - loss: 9.3481 - mean_absolute_error: 1.7321
Epoch 28/50
10381/10381 [==============================] - 5s 493us/step - loss: 8.9707 - mean_absolute_error: 1.7036
Epoch 29/50
10381/10381 [==============================] - 5s 505us/step - loss: 9.1233 - mean_absolute_error: 1.7122
Epoch 30/50
10381/10381 [==============================] - 5s 493us/step - loss: 9.0131 - mean_absolute_error: 1.6878
Epoch 31/50
10381/10381 [==============================] - 5s 505us/step - loss: 8.8797 - mean_absolute_error: 1.6654
Epoch 32/50
10381/10381 [==============================] - 5s 495us/step - loss: 9.1904 - mean_absolute_error: 1.6849
Epoch 33/50
10381/10381 [==============================] - 5s 497us

10381/10381 [==============================] - 5s 505us/step - loss: 9.3196 - mean_absolute_error: 1.6803
Epoch 37/50
10381/10381 [==============================] - 5s 497us/step - loss: 9.2886 - mean_absolute_error: 1.6820
Epoch 38/50
10381/10381 [==============================] - 5s 496us/step - loss: 9.1376 - mean_absolute_error: 1.6636
Epoch 39/50
10381/10381 [==============================] - 5s 496us/step - loss: 9.3697 - mean_absolute_error: 1.6880
Epoch 40/50
10381/10381 [==============================] - 5s 500us/step - loss: 9.0060 - mean_absolute_error: 1.6643
Epoch 41/50
10381/10381 [==============================] - 5s 495us/step - loss: 9.0889 - mean_absolute_error: 1.6602
Epoch 42/50
10381/10381 [==============================] - 5s 496us/step - loss: 9.3453 - mean_absolute_error: 1.6643
Epoch 43/50
10381/10381 [==============================] - 5s 500us/step - loss: 8.8430 - mean_absolute_error: 1.6414
Epoch 44/50
10381/10381 [==============================] - 5s 493us/

In [11]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=2))

Average Kappa score after a 5-fold cross validation:  0.96


In [14]:
# Splitting dataset into training and test set and generating word embeddings for other models other than
# neural networks

indep_train, indep_test, dep_train, dep_test = train_test_split(dataset, scores, test_size = 0.25)

train_essays2 = indep_train['essay']
test_essays2 = indep_test['essay']
    
sentences2 = []


for essay2 in train_essays2:
            # Obtaining all sentences from the training set of essays.
            sentences2 += sentence_tokens(essay2)
            
# Initializing variables for word2vec model.
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

clean_train_essays2 = []
    
# Generate training and testing data word vectors.
for essay_text2 in train_essays2:
    clean_train_essays2.append(word_tokens(essay_text2))
trainDataVecs2 = getAvgFeatureVecs(clean_train_essays2, model, num_features)
    
clean_test_essays2 = []
for essay_text2 in test_essays2:
    clean_test_essays2.append(word_tokens(essay_text2))
testDataVecs2 = getAvgFeatureVecs(clean_test_essays2, model, num_features)
    
trainDataVecs2 = np.array(trainDataVecs2)
testDataVecs2 = np.array(testDataVecs2)

Training Word2Vec Model...


/home/linux-device/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [15]:
# Generating scores using Linear Regression Model

linear_regressor = LinearRegression()

linear_regressor.fit(trainDataVecs2, dep_train)

dep_pred = linear_regressor.predict(testDataVecs2)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(dep_test, dep_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % explained_variance_score(dep_test,dep_pred))

#print('Cohen\'s kappa score: %.2f' % cohen_kappa_score(dep_pred, dep_test))
print("Kappa Score: {0:.2f}".format(cohen_kappa_score(dep_test.values,np.around(dep_pred),weights='quadratic')))

Mean squared error: 20.29
Variance score: 0.75
Kappa Score: 0.86


In [16]:
#Generating scores using Gradient Boosting regressor

'''from sklearn.model_selection import GridSearchCV
params = {'n_estimators':[100, 1000], 'max_depth':[2], 'min_samples_split': [2],
          'learning_rate':[3, 1, 0.1, 0.3], 'loss': ['ls']}

gbr = ensemble.GradientBoostingRegressor()

grid = GridSearchCV(gbr, params)
grid.fit(trainDataVecs2, dep_train)

y_pred = grid.predict(testDataVecs2)

# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_)'''

#USING THE PARAMS FOUND OUT USING GRID SEARCH CV
gbr = ensemble.GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)
gbr.fit(trainDataVecs2, dep_train)
dep_pred = gbr.predict(testDataVecs2)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(dep_test, dep_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % explained_variance_score(dep_test,dep_pred))

print("Kappa Score: {0:.2f}".format(cohen_kappa_score(dep_test.values,np.around(dep_pred),weights='quadratic')))

Mean squared error: 9.58
Variance score: 0.88
Kappa Score: 0.94


In [17]:
svr = SVR()

'''parameters = {'kernel':['linear', 'rbf'], 'C':[1, 100], 'gamma':[0.1, 0.001]}

grid = GridSearchCV(svr, parameters)
grid.fit(trainDataVecs2, dep_train)

y_pred = grid.predict(testDataVecs2)

# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_)'''

#USING THE PARAMS FOUND OUT USING GRID SEARCH CV
svr = SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr.fit(trainDataVecs2, dep_train)
dep_pred = svr.predict(testDataVecs2)


# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(dep_test, dep_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % explained_variance_score(dep_test,dep_pred))

#Cohen's Kappa score
print("Kappa Score: {0:.2f}".format(cohen_kappa_score(dep_test.values,np.around(dep_pred),weights='quadratic')))

Mean squared error: 41.51
Variance score: 0.52
Kappa Score: 0.64


In [45]:
# As lstm outperforms all other models, so using it for predicting the scores for the final dataset
valid_set = pd.read_csv('valid_set.tsv', sep='\t', encoding='ISO-8859-1')

In [46]:
valid_set = valid_set.drop(['domain2_predictionid'], axis = 1)

In [47]:
valid_set.head()

,essay_id,essay_set,essay,domain1_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792


In [50]:
valid_test_essays = valid_set['essay']

In [51]:
valid_test_essays

0       Dear @ORGANIZATION1, @CAPS1 more and more peop...
1       Dear @LOCATION1 Time @CAPS1 me tell you what I...
2       Dear Local newspaper, Have you been spending a...
3       Dear Readers, @CAPS1 you imagine how life woul...
4       Dear newspaper, I strongly believe that comput...
5       Dear local newspaper, @CAPS1 the caveman found...
6       Dear newspaper editor, @CAPS1 now is a letter ...
7       Dear @ORGANIZATION1, @CAPS1, there has been so...
8       Dear Local Newspaper, I would like to complain...
9       Dear Newspaper, @CAPS1 having kids wasting the...
10      Dear @CAPS1 @CAPS2, @CAPS3 should think that c...
11      Dear Local @CAPS1, Computers, and other techno...
12      Why people should use computers? Computers are...
13      @CAPS1, its time for supper! @CAPS2 on, mom I ...
14      Dear local newspaper: I am writing this letter...
15      Effects of computers @CAPS1 local newspaper, H...
16      Dear to whom it @MONTH1 concern, @CAPS1 is an ...
17      Dear @

In [55]:
sentences = []
    
for valid_essay in valid_test_essays:
        sentences += sentence_tokens(valid_essay)
            
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

valid_clean_test_essays = []
    
# Generate training and testing data word vectors.
for essay_text in valid_test_essays:
    valid_clean_test_essays.append(word_tokens(essay_text))
valid_testDataVecs = getAvgFeatureVecs(valid_clean_test_essays, model, num_features)

valid_testDataVecs = np.array(valid_testDataVecs)
# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
valid_testDataVecs = np.reshape(valid_testDataVecs, (valid_testDataVecs.shape[0], 1, valid_testDataVecs.shape[1]))
    
predicted_scores = lstm_model.predict(valid_testDataVecs)
    
# Round y_pred to the nearest integer.
predicted_scores = np.around(predicted_scores)

Training Word2Vec Model...


/home/linux-device/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/linux-device/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [155]:
submission = valid_set.drop(['essay'], axis = 1)

In [156]:
predicted_score = predicted_scores.tolist()

In [157]:
predicted_score = pd.Series([score for sublist in predicted_scores for score in sublist])

In [158]:
predicted_score.head()

0    6.0
1    5.0
2    7.0
3    7.0
4    6.0
dtype: float64

In [159]:
submission = pd.concat([submission, predicted_score], axis = 1).rename(columns = {0:"predicted_score"}).iloc[:,[2,0,1,3]]
submission.to_excel("Submission.xls",index=False)